# Coulomb Energy Functional
## Theory, Discretization and Evaluation
**Scope:** Define the Coulomb energy functional once. Evaluate it for 
  (i) smooth densities, 
  (ii) point charges, 
  (iii) periodic toy lattice (Madelung preview).  

**Goal:** Make $E[\rho]$ operational as $E=\tfrac12 \rho^\mathsf{T}V\rho$ and connect to Poisson and Green’s functions.


## Coulomb Energy Functional
We define the electrostatic energy of a charge distribution as a functional $E_{\mathrm{Coul}}[\rho]$ and show three equivalent forms:
$$E=\tfrac12\int \rho \phi\,d^3r$$

$$E=\tfrac{\varepsilon_0}{2}\int |\mathbf{E}|^2\,d^3r$$
$$E=\tfrac12\iint \rho(\mathbf{r})\,G(\mathbf{r},\mathbf{r}')\,\rho(\mathbf{r}')\,d^3r\,d^3r'$$

We then discretize the functional into a quadratic form $E=\tfrac12\rho^\mathsf{T}V\rho$ and evaluate it numerically.


## Symbols and Units


In [3]:
from dataclasses import dataclass
import numpy as np

class ConstantsSI:
  epsilon0 = 

@dataclass
class ChargeDensity():
  r: np.ndarray
  rho: np.ndarray

@dataclass
class ElectrostaticPotential():
  r: np.ndarray
  phi: np.ndarray

@dataclass
class ElectricField():
  r: np.ndarray
  Efield: np.ndarray


SyntaxError: invalid syntax (104856331.py, line 5)

## Discretized 1d Grid and Laplacian Operator

In [2]:
N = 400
L = 1.0
r = np.linspace(-L/2, L/2, N, endpoint=False)
dr = r[1]-r[0]

# NOTE: This dense form is for pedagogy; for large N, use sparse matrices.
e = np.ones(N)
Lap = (np.diag(-2*e) + np.diag(e[:-1], 1) + np.diag(e[:-1], -1)) / dx**2


NameError: name 'np' is not defined

# Notebook Sketch: Coulomb Energy Functional (Theory → Discretization → Lattice)

**Filename:** `coulomb_energy_functional.ipynb`  
**Scope:** Define the Coulomb energy functional once. Evaluate it for (i) smooth densities, (ii) point charges, (iii) periodic toy lattice (Madelung preview).  
**Goal:** Make $E[\rho]$ operational as $E=\tfrac12 \rho^\mathsf{T}V\rho$ and connect to Poisson and Green’s functions.

---

## Cell 0 — Title + learning objectives (Markdown)

# Coulomb Energy Functional
We define the electrostatic energy of a charge distribution as a functional $E_{\mathrm{Coul}}[\rho]$ and show three equivalent forms:
1. $E=\tfrac12\int \rho \phi\,d^3r$
2. $E=\tfrac{\varepsilon_0}{2}\int |\mathbf{E}|^2\,d^3r$
3. $E=\tfrac12\iint \rho(\mathbf{r})\,G(\mathbf{r},\mathbf{r}')\,\rho(\mathbf{r}')\,d^3r\,d^3r'$

We then discretize the functional into a quadratic form $E=\tfrac12\rho^\mathsf{T}V\rho$ and evaluate it numerically.

---

## Cell 1 — Authoritative source (Markdown)

**Primary reference (authoritative):** J. D. Jackson, *Classical Electrodynamics*, 3rd ed.  
Electrostatic energy of charge distributions and equivalent field forms are standard results in the early electrostatics chapters (Poisson/Green’s functions, energy expressions).

**Secondary (solid-state bridge):** N. W. Ashcroft and N. D. Mermin, *Solid State Physics*  
Electrostatic energy in ionic crystals leading to Madelung energy.

**Electronic-structure bridge:** R. M. Martin, *Electronic Structure: Basic Theory and Practical Methods*  
Hartree energy as the Coulomb functional evaluated on electron density.

---

## Cell 2 — Symbols and units (Markdown)

Let:
- charge density: $\rho(\mathbf{r})$ in C/m$^3$
- potential: $\phi(\mathbf{r})$ in V
- electric field: $\mathbf{E}(\mathbf{r})=-\nabla \phi$
- vacuum permittivity: $\varepsilon_0$
- Green’s function (free space): $G(\mathbf{r},\mathbf{r}')=\dfrac{1}{4\pi|\mathbf{r}-\mathbf{r}'|}$

---

## Cell 3 — Core theory: Poisson + energy functional (Markdown)

Poisson equation:
$$
\nabla^2 \phi(\mathbf{r})=-\frac{\rho(\mathbf{r})}{\varepsilon_0}.
$$

Coulomb energy functional:
$$
E_{\mathrm{Coul}}[\rho]=\frac12\int \rho(\mathbf{r})\,\phi(\mathbf{r})\,d^3r.
$$

Using Green’s function:
$$
\phi(\mathbf{r})=\frac{1}{\varepsilon_0}\int G(\mathbf{r},\mathbf{r}')\,\rho(\mathbf{r}')\,d^3r',
\qquad
G(\mathbf{r},\mathbf{r}')=\frac{1}{4\pi|\mathbf{r}-\mathbf{r}'|}.
$$

Therefore:
$$
E_{\mathrm{Coul}}[\rho]
=
\frac12\iint
\frac{\rho(\mathbf{r})\,\rho(\mathbf{r}')}{4\pi\varepsilon_0|\mathbf{r}-\mathbf{r}'|}
\,d^3r\,d^3r'.
$$

---

## Cell 4 — Equivalent field-energy form (Markdown)

Using $\mathbf{E}=-\nabla\phi$ and integration by parts (with suitable boundary conditions),
$$
E_{\mathrm{Coul}}[\rho]
=
\frac{\varepsilon_0}{2}\int |\mathbf{E}(\mathbf{r})|^2\,d^3r.
$$

This form is numerically useful when computing $\mathbf{E}$ from $\phi$.

---

## Cell 5 — Setup: imports (Code)

import numpy as np
import matplotlib.pyplot as plt

---

## Cell 6 — 1D toy model disclaimer (Markdown)

We will implement a **1D analogue** to make discretization transparent.
We replace the 3D kernel by an appropriate 1D Green’s function or by solving a 1D Poisson problem on a finite interval with chosen boundary conditions.

The algebraic structure (functional → quadratic form) is the same.

---

## Cell 7 — Discretization as basis expansion (Markdown)

Choose grid points $x_i$ and basis states $|i\rangle$.
Represent:
- density vector: $\rho_i \approx \rho(x_i)$
- potential vector: $\phi_i \approx \phi(x_i)$

Then the discrete functional becomes:
$$
E \approx \frac12 \sum_i \rho_i \phi_i \,\Delta x
=
\frac12\,\rho^\mathsf{T}(\Delta x\,I)\,\phi.
$$

If $\phi = V\rho$ for some discrete Coulomb operator $V$, then:
$$
E = \frac12\,\rho^\mathsf{T}(\Delta x\,I)\,V\,\rho.
$$

Define $W \equiv (\Delta x\,I)V$:
$$
E=\frac12\,\rho^\mathsf{T}W\rho.
$$

---

## Cell 8 — Build a discrete Poisson operator (Code)

# grid
N = 400
L = 1.0
x = np.linspace(-L/2, L/2, N, endpoint=False)
dx = x[1] - x[0]

# 1D Laplacian with Dirichlet BC (interior operator)
# (for teaching clarity; later you can swap BCs)
e = np.ones(N)
Lap = (np.diag(-2*e) + np.diag(e[:-1], 1) + np.diag(e[:-1], -1)) / dx**2

---

## Cell 9 — Define a smooth charge density (Code)

sigma = 0.05
rho = np.exp(-(x/sigma)**2)
rho = rho - rho.mean()   # enforce neutrality for stability in simple solvers

---

## Cell 10 — Solve Poisson for phi (Code)

# Solve Lap phi = -rho/eps0 in arbitrary units with eps0=1 for toy model
eps0 = 1.0
rhs = -rho/eps0

# naive solve (dense); later upgrade to sparse
phi = np.linalg.solve(Lap, rhs)

---

## Cell 11 — Compute E = 1/2 ∫ rho phi dx (Code)

E_rhophi = 0.5 * np.sum(rho * phi) * dx
E_rhophi

---

## Cell 12 — Visualize rho and phi (Code)

plt.figure()
plt.plot(x, rho, label="rho(x)")
plt.plot(x, phi, label="phi(x)")
plt.legend()
plt.xlabel("x")
plt.show()

---

## Cell 13 — Matrix form E = 1/2 rho^T W rho (Markdown + Code)

We can view $\phi = V\rho$ with $V \equiv -(\varepsilon_0 \nabla^2)^{-1}$ (discretized).  
Thus $W = (\Delta x I)V$ and:
$$
E=\frac12\rho^\mathsf{T}W\rho.
$$

# build V from Laplacian inverse (toy; dense)
V = -np.linalg.inv(Lap) / eps0
W = dx * V

E_quad = 0.5 * rho @ W @ rho
E_quad, E_rhophi

---

## Cell 14 — Point charges as discrete rho (Markdown)

A set of point charges $q_i$ at positions $x_i$ corresponds to:
$$
\rho(x)=\sum_k q_k\,\delta(x-x_k).
$$

On a grid, this becomes “depositing” charge into nearest grid bins:
$\rho_i \propto q_k/\Delta x$ at the nearest $i$ to $x_k$.

Then:
$$
E = \frac12\sum_{i\neq j}\frac{q_i q_j}{4\pi\varepsilon_0|x_i-x_j|}
$$
with self-energy excluded.

---

## Cell 15 — Implement two point charges (Code)

rho_pc = np.zeros_like(x)
q1, q2 = 1.0, -1.0
i1, i2 = N//2 - 40, N//2 + 40
rho_pc[i1] = q1/dx
rho_pc[i2] = q2/dx
rho_pc = rho_pc - rho_pc.mean()

phi_pc = np.linalg.solve(Lap, -rho_pc/eps0)
E_pc = 0.5 * np.sum(rho_pc * phi_pc) * dx
E_pc

---

## Cell 16 — Periodicity preview (Markdown)

For a periodic lattice, naive Coulomb sums are conditionally convergent.  
A correct evaluation requires regularization (Ewald summation or equivalent).

Outcome in ionic solids:
$$
E_{\mathrm{Madelung}}
=
- \frac{z_+z_-e^2}{4\pi\varepsilon_0 r_0}\,M.
$$

In a later notebook:
- implement a toy 1D Ewald-like split,
- extract an effective “Madelung constant” for the model lattice,
- compare convergence with and without screening.

---

## Cell 17 — Exercises (Markdown)

1. Change $\rho(x)$ and verify $E=\tfrac12\int\rho\phi\,dx$ remains consistent with $E=\tfrac12\rho^\mathsf{T}W\rho$.  
2. Replace Dirichlet BC with periodic BC and observe the neutrality constraint.  
3. Implement a screened kernel $G_\kappa(r)=\exp(-\kappa r)/r$ and show improved convergence in lattice sums.  
4. For a simple alternating-charge 1D lattice, compute partial sums and show order dependence, motivating Ewald methods.

---


## References
[^1] J. D. Jackson, *Classical Electrodynamics*, 3rd ed.  
Electrostatic energy of charge distributions and equivalent field forms are standard results in the early electrostatics chapters (Poisson/Green’s functions, energy expressions).

[^2] N. W. Ashcroft and N. D. Mermin, *Solid State Physics*  
Electrostatic energy in ionic crystals leading to Madelung energy.

[^3] R. M. Martin, *Electronic Structure: Basic Theory and Practical Methods*  
Hartree energy as the Coulomb functional evaluated on electron density.
